# Byte-Pair Encoding

Byte-Pair Encoding, is a data compression algorithm that iteratively replaces the most frequent pair of bytes in a sequence with a single, unused byte.

`For example:` Let 'aaabdaaabac' be the string. Here 'aa' is the most frequent pair of bytes, so we replace it with some unused byte, let's say 'Z'. Now the string becomes 'ZabdZabac'. 'ab' is now the most frequent pair, we replace it with 'Y' and get 'ZYdZYac'. After the latest update the most frequent pair be 'ZY'. Replace it with 'X' so the obtained string be 'XdXac'.

To adapt this idea for word segmentation, instead of replacing frequent pair of bytes, we now merge subword pairs that frequently occur. Let's elaborate it stepwise:

- We initialize the vocabulary representing each word as a sequence of characters, plus a special end-of-word symbol '/w', which allows us to restore the original tokenization after translation. More on this below.
- Next, we iteratively count all symbol pairs and merge each occurrence of the most frequent pair (a, b) into ab. Each merge operation produces a new symbol which represents a character n-gram.
- We'll keep repeating the previous process until the target vocabulary size is reached.

Why is end of word symbol '/w' important? Let's say there is a token 'est'. Both the words 'eastern' and 'smallest' contain the subword 'est'. However, meanings of the subword between the two are quite different. With the end of word symbol, if there is a token est/w, the model immediately interprets the subword represents the terminating meaning of the word.

![](./../assets/tokenization/bpe.jpg)
*[[Image Source]](https://www.computer.org/csdl/journal/tb/2020/05/08678449/1nJsrGwiJqg)*

We will be implementing Byte-Pair Encoding using Google's SentencePiece library. Let's get started by installing the library.

`pip3 install sentencepiece`

`NOTE:` BPE Tokenization can be implemented using HuggingFace's tokenizers library as done for WordPiece and Unigram Tokenization. You may refer to `wordpiece.ipynb` or `unigram.ipynb` for similar implementation of BPE Tokenization.

`Remember:`
- To download the dataset and copy it to the correct location.
- Create a 'models' directory inside the root directory.

### Training and loading model

In [1]:
# This code allows you to import '.py' file from one directory behind (i.e. root directory)
import sys
import os

root_dir = os.path.dirname(os.getcwd())
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [2]:
import config
from sentencepiece import SentencePieceTrainer, SentencePieceProcessor

params = ' '.join([
    f'--input=./../{config.DATA_PATH}',
    '--model_type=bpe',
    f'--model_prefix=./../{config.MODEL_PATH}/bpe',
    f'--vocab_size={config.VOCAB_SIZE}',
    '--pad_id=0',
    '--unk_id=1',
    '--bos_id=2',
    '--eos_id=3'
])
SentencePieceTrainer.train(params)

sp = SentencePieceProcessor()
sp.load(f'./../{config.MODEL_PATH}/bpe.model')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=./../data/shakespeare.txt --model_type=bpe --model_prefix=./../models/bpe --vocab_size=1000 --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./../data/shakespeare.txt
  input_format: 
  model_prefix: ./../models/bpe
  model_type: BPE
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2

True

## Showcasing Subword Operations

In [3]:
text = "Good muffins cost $3.88. Please buy me two of them.\n\nThanks.🙂😍"

In [4]:
print('Number of Unique Tokens: {}'.format(sp.get_piece_size()))

Number of Unique Tokens: 1000


In [5]:
encoded_pieces = sp.encode_as_pieces(text)
print(encoded_pieces)

decoded_pieces = sp.decode_pieces(encoded_pieces)
print(decoded_pieces)

['▁G', 'ood', '▁mu', 'ff', 'in', 's', '▁c', 'ost', '▁', '$', '3', '.', '88', '.', '▁P', 'le', 'ase', '▁b', 'u', 'y', '▁me', '▁two', '▁of', '▁them', '.', '▁Than', 'ks', '.', '🙂😍']
Good muffins cost $3.88. Please buy me two of them. Thanks.🙂😍


### Showcasing Subword Operations with Numeric Ids

In [6]:
encoded_ids = sp.encode_as_ids(text)
print(encoded_ids)

decoded_ids = sp.decode_ids(encoded_ids)
print(decoded_ids)

[197, 172, 217, 750, 9, 936, 30, 260, 930, 1, 990, 949, 1, 949, 116, 77, 468, 14, 942, 944, 82, 714, 41, 303, 949, 781, 647, 949, 1]
Good muffins cost  ⁇ 3. ⁇ . Please buy me two of them. Thanks. ⁇ 


In [7]:
piece_id = sp.piece_to_id('▁The')
print(piece_id)

piece = sp.id_to_piece(piece_id)
print(piece)

96
▁The
